In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction import DictVectorizer
import seaborn as sns
import random
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='whitegrid')

In [6]:
df = pd.read_pickle('/Users/serenazhang/Desktop/personalization project/project/Part II/df_all')

In [7]:
df.loc[df.ztarget == 0, ('ztarget')] = -1

In [8]:
def popular_user_items(dataframe,min_user_per_item_thr,min_item_per_user_thr):
    min_user_per_item=0
    min_item_per_user=0
    count=1
    while min_user_per_item<min_user_per_item_thr and min_item_per_user<min_item_per_user_thr:

        print("**iteration",count,"starts!**")
        item_count=dataframe.groupby("song_id").count().reset_index()
        min_user_per_item=item_count["msno"].min()
        item_qualify=item_count[item_count["msno"]>=min_user_per_item_thr]
        item_list=item_qualify["song_id"]
        print("num of qualified items:", len(item_list))

        user_count=dataframe.groupby("msno").count().reset_index()
        min_item_per_user=user_count["song_id"].min()
        user_qualify=user_count[user_count["song_id"]>=min_item_per_user_thr]
        user_list=user_qualify["msno"]
        print("num of qualified users:", len(user_list))
        
        dataframe=dataframe.loc[(dataframe['song_id'].isin(item_list))&(dataframe["msno"].isin(user_list))]
        count+=1
   
    return dataframe


In [27]:
popular= popular_user_items(df,50,50)

**iteration 1 starts!**
num of qualified items: 17878
num of qualified users: 20543
**iteration 2 starts!**
num of qualified items: 17635
num of qualified users: 19357
**iteration 3 starts!**
num of qualified items: 17385
num of qualified users: 19341
**iteration 4 starts!**
num of qualified items: 17381
num of qualified users: 19321
**iteration 5 starts!**
num of qualified items: 17378
num of qualified users: 19320
**iteration 6 starts!**
num of qualified items: 17376
num of qualified users: 19320


In [31]:
popular.shape

(5571598, 12)

In [32]:
def sampling(data,perc_range):
    item_set=data.song_id.unique().tolist()
    user_set=data.msno.unique().tolist()
    samples=[]
    #construct a list of samples
    for i in perc_range:
        sample_from_user=random.sample(user_set, int(i*len(user_set)))
        df_sample=data.loc[(data['msno'].isin(sample_from_user))]
        samples.append(df_sample)
    return samples

In [44]:
popular_list=sampling(popular,[0.065,0.1,0.25,0.5])
normal_list=sampling(df,[.05,0.1,0.25,0.50])

In [45]:
df_normal=normal_list[0]
df_popular=popular_list[0]

In [46]:
df_normal.shape

(360744, 12)

In [47]:
df_popular.shape

(363375, 12)

In [93]:
df_normal.to_pickle('/Users/serenazhang/Desktop/personalization project/project/Part II/df_normal')
df_popular.to_pickle('/Users/serenazhang/Desktop/personalization project/project/Part II/df_popular')